# Inventory Monitoring at Distribution Centers

This notebook guides you through building, training, and deploying a machine learning model for **Inventory Monitoring at Distribution Centers** using **AWS SageMaker**. The goal is to create a model that can count the number of objects in bins based on images. This project uses the **Amazon Bin Image Dataset**, which contains images from Amazon Fulfillment Centers showing bins with various objects. 

The key steps in this project include:
1. Data preparation and upload to S3.
2. Model training on SageMaker using a custom script (`train.py`).
3. Model deployment to a SageMaker endpoint for inference.
4. Optional tasks such as hyperparameter tuning, debugging, profiling, and cost analysis.

This project focuses on implementing a machine learning engineering pipeline rather than achieving high accuracy.


In [1]:
# TODO: Install any packages that you might need
!pip install tqdm boto3

In [2]:
# TODO: Import any packages that you might need
import os
import json
import boto3
import sagemaker
from sagemaker import get_execution_role
from tqdm import tqdm

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


## Data Preparation
**TODO:** Run the cell below to download the data.

The cell below creates a folder called `train_data`, downloads training data and arranges it in subfolders. Each of these subfolders contain images where the number of objects is equal to the name of the folder. For instance, all images in folder `1` has images with 1 object in them. Images are not divided into training, testing or validation sets. If you feel like the number of samples are not enough, you can always download more data (instructions for that can be found [here](https://registry.opendata.aws/amazon-bin-imagery/)). However, we are not acessing you on the accuracy of your final trained model, but how you create your machine learning engineering pipeline.

In [3]:
#import os
#import json
#import boto3

def download_and_arrange_data():
    s3_client = boto3.client('s3')

    with open('file_list.json', 'r') as f:
        d=json.load(f)

    for k, v in d.items():
        print(f"Downloading Images with {k} objects")
        directory=os.path.join('train_data', k)
        if not os.path.exists(directory):
            os.makedirs(directory)
        for file_path in tqdm(v):
            file_name=os.path.basename(file_path).split('.')[0]+'.jpg'
            s3_client.download_file('aft-vbi-pds', os.path.join('bin-images', file_name),
                             os.path.join(directory, file_name))

download_and_arrange_data()

100%|██████████| 1228/1228 [01:36<00:00, 12.77it/s]


100%|██████████| 2299/2299 [03:05<00:00, 12.38it/s]


100%|██████████| 2666/2666 [03:33<00:00, 12.46it/s]


100%|██████████| 2373/2373 [03:08<00:00, 12.56it/s]


100%|██████████| 1875/1875 [02:25<00:00, 12.91it/s]


## Dataset

For this project, we are using the **Amazon Bin Image Dataset**, which contains images of bins from Amazon Fulfillment Centers. Each image shows a bin with one or more items inside, where items are placed randomly. This dataset allows us to build a machine learning model that can classify images based on the number of objects in each bin, which is essential for efficient inventory tracking and management in distribution centers.

### Structure of the Dataset
Each image in the dataset is associated with a metadata file containing details about the items in the bin. The key information for our project is the `EXPECTED_QUANTITY`, which indicates the number of objects present in each bin. This quantity ranges from **1 to 5 objects**, and we use it to create labeled data for training a classification model with five classes:
- **Class 1**: Images with 1 object in the bin
- **Class 2**: Images with 2 objects in the bin
- **Class 3**: Images with 3 objects in the bin
- **Class 4**: Images with 4 objects in the bin
- **Class 5**: Images with 5 objects in the bin

### Data Preprocessing
For this project, the images have been organized into folders by the number of objects (1–5), based on the `EXPECTED_QUANTITY` field in the metadata. We then split the data into training, validation, and test sets to facilitate model evaluation and ensure generalization.

This dataset helps us train a classification model to determine the number of items in a bin from an image, enabling automated inventory counting at distribution centers.

More information about the dataset can be found [here](https://registry.opendata.aws/amazon-bin-imagery/).

This function splits the data into `train`, `validation`, and `test` sets based on the specified ratios. The output directory `processed_data` will contain subdirectories for training, validation, and test sets, each further organized by object count.

In [4]:
#TODO: Perform any data cleaning or data preprocessing
import shutil
from sklearn.model_selection import train_test_split

# Split data into train, validation, and test sets
def split_data(data_dir, output_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(os.path.join(output_dir, 'train'))
    os.makedirs(os.path.join(output_dir, 'validation'))
    os.makedirs(os.path.join(output_dir, 'test'))
    
    for object_count in os.listdir(data_dir):
        images = os.listdir(os.path.join(data_dir, object_count))
        train, temp = train_test_split(images, train_size=train_ratio)
        val, test = train_test_split(temp, train_size=val_ratio/(val_ratio + test_ratio))
        
        for subset, subset_images in zip(['train', 'validation', 'test'], [train, val, test]):
            subset_dir = os.path.join(output_dir, subset, object_count)
            os.makedirs(subset_dir, exist_ok=True)
            for image in subset_images:
                shutil.copy2(os.path.join(data_dir, object_count, image), subset_dir)
        print(f"Data split for category '{object_count}' complete.")

# Run data split
split_data('train_data', 'processed_data')


Data split for category '1' complete.
Data split for category '2' complete.
Data split for category '3' complete.
Data split for category '4' complete.
Data split for category '5' complete.


In [13]:
#TODO: Upload the data to AWS S3

import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch
#import boto3
#import os

session = sagemaker.Session()

bucket=session.default_bucket()
print("Default Bucket: {}".format(bucket))

region = session.boto_region_name
print("AWS Region: {}".format(region))

role = get_execution_role() #sagemaker iam role
print("RoleArn: {}".format(role))

data = "s3://{}/{}/".format(bucket, "inventory-monitoring")
print(f"Uploading data to S3 at {data}")
output = "s3://{}/{}/".format(bucket, "output")
model_dir = "s3://{}/{}/".format(bucket, "model")
os.environ["DEFAULT_S3_BUCKET"] =bucket
os.environ['SM_CHANNEL_TRAIN']=data 
os.environ['SM_OUTPUT_DATA_DIR']=output
os.environ['SM_MODEL_DIR']=model_dir

Default Bucket: sagemaker-us-east-1-254792028129
AWS Region: us-east-1
RoleArn: arn:aws:iam::254792028129:role/service-role/AmazonSageMaker-ExecutionRole-20241116T170316
Uploading data to S3 at s3://sagemaker-us-east-1-254792028129/inventory-monitoring/


In [14]:
s3_data_path = session.upload_data(path='processed_data', bucket=bucket, key_prefix=f'inventory-monitoring/data')

In [4]:
output

's3://sagemaker-us-east-1-254792028129/output/'

In [9]:
#s3_data_path=data

## Model Training
**TODO:** This is the part where you can train a model. The type or architecture of the model you use is not important. 

**Note:** You will need to use the `train.py` script to train your model.

`train.py`, is simple model, while `benchmark.py` is our benchmark model which is ResNet50.


### Simple Model

In [18]:
#TODO: Declare your model training hyperparameter.
#NOTE: You do not need to do hyperparameter tuning. You can use fixed hyperparameter values
# Define hyperparameters for the training job
hyperparameters = {
    "batch-size": 64,
    "epochs": 5,
    "lr": 0.005
}


In [20]:
#TODO: Create your training estimator
# Import the PyTorch estimator
from sagemaker.pytorch import PyTorch

# Create a PyTorch Estimator
estimator = PyTorch(
    entry_point="train.py",                 # Path to the training script
    role=role,                              # IAM role with permissions
    instance_count=1,                       # Number of instances
    instance_type="ml.p3.2xlarge", #"ml.m5.xlarge",           # Instance type for training
    framework_version="1.8",                # PyTorch version
    py_version="py3",                       # Python version
    hyperparameters=hyperparameters,        # Hyperparameters defined above
    output_path=output                      # Path for saving output artifacts
)


In [22]:
# TODO: Fit your estimator
# Start the training job
estimator.fit({"train": s3_data_path}, job_name="simple-train2")

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: simple-train2


2024-11-17 15:53:46 Starting - Starting the training job
2024-11-17 15:53:46 Pending - Training job waiting for capacity...
2024-11-17 15:54:21 Pending - Preparing the instances for training......
2024-11-17 15:55:11 Downloading - Downloading input data...
2024-11-17 15:55:41 Downloading - Downloading the training image........................
2024-11-17 15:59:54 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-11-17 16:00:06,809 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-11-17 16:00:06,841 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-11-17 16:00:06,844 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-11-17 16:00:07,134 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_p

### Benchmark Model

using `benchmark.py`

In [24]:
hyperparameters = {
    "batch-size": 64,
    "epochs": 5,
    "lr": 0.005
}

from sagemaker.pytorch import PyTorch

# Create a PyTorch Estimator
estimator = PyTorch(
    entry_point="benchmark.py",                 # Path to the training script
    role=role,                              # IAM role with permissions
    instance_count=1,                       # Number of instances
    instance_type="ml.p3.2xlarge", #"ml.m5.xlarge",           # Instance type for training
    framework_version="1.8",                # PyTorch version
    py_version="py3",                       # Python version
    hyperparameters=hyperparameters,        # Hyperparameters defined above
    output_path=output                      # Path for saving output artifacts
)

# Start the training job
estimator.fit({"train": s3_data_path},job_name="benchmark-train2")

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: benchmark-train2


2024-11-17 16:08:25 Starting - Starting the training job...
2024-11-17 16:08:55 Pending - Training job waiting for capacity...
2024-11-17 16:09:18 Pending - Preparing the instances for training...
2024-11-17 16:09:54 Downloading - Downloading input data...
2024-11-17 16:10:24 Downloading - Downloading the training image........................
2024-11-17 16:14:27 Training - Training image download completed. Training in progress...bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-11-17 16:14:43,136 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-11-17 16:14:43,167 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-11-17 16:14:43,170 sagemaker_pytorch_container.training INFO     Invoking user training script.
2024-11-17 16:14:43,453 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_p

## Standout Suggestions
You do not need to perform the tasks below to finish your project. However, you can attempt these tasks to turn your project into a more advanced portfolio piece.

### Hyperparameter Tuning
**TODO:** Here you can perform hyperparameter tuning to increase the performance of your model. You are encouraged to 
- tune as many hyperparameters as you can to get the best performance from your model
- explain why you chose to tune those particular hyperparameters and the ranges.


In [25]:
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)

In [26]:
#TODO: Create your hyperparameter search space
hyperparameter_ranges = {
    "lr": ContinuousParameter(0.005, 0.05), #CategoricalParameter,
    "batch-size": CategoricalParameter([32, 64]),
    "epochs": IntegerParameter(5, 7)
}

objective_metric_name = "Test Loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "Test Loss", "Regex": "Testing Loss: ([0-9\\.]+)"}]

In [27]:
#TODO: Create your training estimator


estimator = PyTorch(
    entry_point="hpo.py",
    role=get_execution_role(),
    py_version='py36',
    framework_version="1.8",
    instance_count=1,
    instance_type='ml.p3.2xlarge'
)

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=2,
    max_parallel_jobs=1,# due to AWS limit, I used 1 instead of 2 or more.
    objective_type=objective_type,
    early_stopping_type="Auto"
)

In [28]:
# TODO: Fit your estimator
tuner.fit({"train": s3_data_path}, job_name="hpo-train2")

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating hyperparameter tuning job with name: hpo-train2


......................................................................................................................................................................................................................................................................!


In [29]:
# TODO: Find the best hyperparameters

best_estimator = tuner.best_estimator()

# Get the hyperparameters of the best-trained model
best_estimator.hyperparameters()


2024-11-17 16:37:19 Starting - Starting the training job
2024-11-17 16:37:19 Pending - Preparing the instances for training
2024-11-17 16:37:19 Downloading - Downloading the training image
2024-11-17 16:37:19 Training - Training image download completed. Training in progress.
2024-11-17 16:37:19 Uploading - Uploading generated training model
2024-11-17 16:37:19 Completed - Resource reused by training job: hpo-train2-002-ca095af7


{'_tuning_objective_metric': '"Test Loss"',
 'batch-size': '"64"',
 'epochs': '5',
 'lr': '0.02852576380758002',
 'sagemaker_container_log_level': '20',
 'sagemaker_estimator_class_name': '"PyTorch"',
 'sagemaker_estimator_module': '"sagemaker.pytorch.estimator"',
 'sagemaker_job_name': '"hpo-train2"',
 'sagemaker_program': '"hpo.py"',
 'sagemaker_region': '"us-east-1"',
 'sagemaker_submit_directory': '"s3://sagemaker-us-east-1-254792028129/hpo-train2/source/sourcedir.tar.gz"'}

In [30]:
print('batch size '+best_estimator.hyperparameters()['batch-size'])
print('epochs '+best_estimator.hyperparameters()['epochs'])
print('learning rate '+best_estimator.hyperparameters()['lr'])

batch size "64"
epochs 5
learning rate 0.02852576380758002


### Model Deploying and Querying
**TODO:** Can you deploy your model to an endpoint and then query that endpoint to get a result?
We will deploy 3 models.

* simple-train2
* benchmark-train2:
* hypo3-train: best from them

The function deploy_model_benchmark(...) only used by benchmark-train2



In [48]:
from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor


jpeg_serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")
json_deserializer = sagemaker.deserializers.JSONDeserializer()


class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )

In [49]:
# TODO: Deploy your model to an endpoint

def deploy_model(endpoint_name, model_tar_path):
    # Define the model
    pytorch_model = PyTorchModel(
        model_data= model_tar_path,
        role=role,
        sagemaker_session=session,
        entry_point="inference.py",  # Path to the inference script if needed
        framework_version="1.8",
        py_version="py36",
        predictor_cls=ImagePredictor
    )
    
    # Deploy the model to an endpoint
    predictor = pytorch_model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.xlarge",
        endpoint_name=endpoint_name
    )
    
    print(f"Model deployed to endpoint: {predictor.endpoint_name}")

In [50]:
# Simple Model Path
trained_model_s3_path = "s3://sagemaker-us-east-1-254792028129/output/"
model_tar_path = f"{trained_model_s3_path}simple-train2/output/model.tar.gz"
endpoint_name="endpoint-simple"
deploy_model(endpoint_name, model_tar_path)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-254792028129/output/simple-train2/output/model.tar.gz), script artifact (None), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-254792028129/pytorch-inference-2024-11-17-23-06-02-292/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-inference-2024-11-17-23-06-13-153
INFO:sagemaker:Creating endpoint-config with name endpoint-simple
INFO:sagemaker:Creating endpoint with name endpoint-simple


-------!Model deployed to endpoint: endpoint-simple


In [52]:
# Benchmark Model Path
def deploy_model_benchmark(endpoint_name, model_tar_path):
    # Define the model
    pytorch_model = PyTorchModel(
        model_data= model_tar_path,
        role=role,
        sagemaker_session=session,
        entry_point="inference_benchmark.py",  # Path to the inference script if needed
        framework_version="1.8",
        py_version="py36",
        predictor_cls=ImagePredictor
    )
    
    # Deploy the model to an endpoint
    predictor = pytorch_model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.xlarge",
        endpoint_name=endpoint_name
    )
    
    print(f"Model deployed to endpoint: {predictor.endpoint_name}")
trained_model_s3_path = "s3://sagemaker-us-east-1-254792028129/output/"
model_tar_path = f"{trained_model_s3_path}benchmark-train2/output/model.tar.gz"
endpoint_name="endpoint-resnet"
deploy_model_benchmark(endpoint_name, model_tar_path)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-254792028129/output/benchmark-train2/output/model.tar.gz), script artifact (None), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-254792028129/pytorch-inference-2024-11-17-23-12-19-002/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-inference-2024-11-17-23-12-27-617
INFO:sagemaker:Creating endpoint-config with name endpoint-resnet
INFO:sagemaker:Creating endpoint with name endpoint-resnet


------!Model deployed to endpoint: endpoint-resnet


In [47]:
# Tunned Model Path
def deploy_model_hpo(endpoint_name, model_tar_path):
    # Define the model
    pytorch_model = PyTorchModel(
        model_data= model_tar_path,
        role=role,
        sagemaker_session=session,
        entry_point="inference_hpo.py",  # Path to the inference script if needed
        framework_version="1.8",
        py_version="py36",
        predictor_cls=ImagePredictor
    )
    
    # Deploy the model to an endpoint
    predictor = pytorch_model.deploy(
        initial_instance_count=1,
        instance_type="ml.m5.xlarge",
        endpoint_name=endpoint_name
    )
    
    print(f"Model deployed to endpoint: {predictor.endpoint_name}")
trained_model_s3_path = "s3://sagemaker-us-east-1-254792028129/" #"s3://sagemaker-us-east-1-254792028129/output/"
model_tar_path = f"{trained_model_s3_path}hpo-train2-001-369f35fa/output/model.tar.gz"
endpoint_name="endpoint-hpo"
deploy_model_hpo(endpoint_name, model_tar_path)

INFO:sagemaker:Repacking model artifact (s3://sagemaker-us-east-1-254792028129/hpo-train2-001-369f35fa/output/model.tar.gz), script artifact (None), and dependencies ([]) into single tar.gz file located at s3://sagemaker-us-east-1-254792028129/pytorch-inference-2024-11-17-19-41-13-799/model.tar.gz. This may take some time depending on model size...
INFO:sagemaker:Creating model with name: pytorch-inference-2024-11-17-19-41-24-330
INFO:sagemaker:Creating endpoint-config with name endpoint-hpo
INFO:sagemaker:Creating endpoint with name endpoint-hpo


------!Model deployed to endpoint: endpoint-hpo


In [53]:
import os
import io
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sagemaker.predictor import Predictor
from torchvision.datasets import ImageFolder
from torchvision import transforms
from PIL import Image

# Preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define valid classes
valid_classes = [1, 2, 3, 4, 5]

# Initialize results storage
results = {}

# List of endpoints and their names
endpoints = {
    "endpoint-resnet": Predictor(endpoint_name="endpoint-resnet"),
    "endpoint-simple": Predictor(endpoint_name="endpoint-simple"),
    "endpoint-hpo": Predictor(endpoint_name="endpoint-hpo")
}

# Iterate through endpoints
for endpoint_name, predictor in endpoints.items():
    print(f"Evaluating {endpoint_name}...")

    true_labels = []
    predictions = []

    # Iterate over all test data
    for class_label in valid_classes:  # Folders 1, 2, ..., 5
        folder_path = os.path.join("./processed_data/test", str(class_label))
        if not os.path.isdir(folder_path):
            continue

        for image_file in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_file)
            if not os.path.isfile(image_path):
                continue

            # Open and preprocess the image
            with open(image_path, "rb") as f:
                payload = f.read()

            # Make prediction
            response = predictor.predict(payload, initial_args={"ContentType": "image/jpeg"})
            predicted_class = int(np.array(eval(response.decode()))[0]) + 1  # Add 1 to align with valid_classes
            
            # Append true label and prediction
            true_labels.append(class_label)
            predictions.append(predicted_class)

    # Validate class range
    print(f"Unique true labels: {set(true_labels)}")
    print(f"Unique predicted labels: {set(predictions)}")

    # Compute metrics
    accuracy = accuracy_score(true_labels, predictions)
    conf_matrix = confusion_matrix(true_labels, predictions, labels=valid_classes)
    classification_report_str = classification_report(
        true_labels, predictions, labels=valid_classes, target_names=[str(c) for c in valid_classes]
    )

    # Print results
    print(f"Accuracy for {endpoint_name}: {accuracy * 100:.2f}%")
    print(f"Confusion Matrix for {endpoint_name}:\n{conf_matrix}")
    print(f"Classification Report for {endpoint_name}:\n{classification_report_str}")

    # Store results
    results[endpoint_name] = {
        "accuracy": accuracy,
        "confusion_matrix": conf_matrix,
        "classification_report": classification_report_str
    }

# Print final summary
print("\nFinal Results Summary:")
for endpoint_name, metrics in results.items():
    print(f"Endpoint: {endpoint_name}")
    print(f"Accuracy: {metrics['accuracy'] * 100:.2f}%")
    print(f"Confusion Matrix:\n{metrics['confusion_matrix']}")
    print(f"Classification Report:\n{metrics['classification_report']}\n")


Evaluating endpoint-resnet...
Unique true labels: {1, 2, 3, 4, 5}
Unique predicted labels: {1, 2, 3, 4, 5}
Accuracy for endpoint-resnet: 27.53%
Confusion Matrix for endpoint-resnet:
[[ 36   3  34  42   8]
 [ 11   5  46 112  56]
 [  5   1  40 132  89]
 [  2   1  33 120  82]
 [  2   0  24  75  87]]
Classification Report for endpoint-resnet:
              precision    recall  f1-score   support

           1       0.64      0.29      0.40       123
           2       0.50      0.02      0.04       230
           3       0.23      0.15      0.18       267
           4       0.25      0.50      0.33       238
           5       0.27      0.46      0.34       188

    accuracy                           0.28      1046
   macro avg       0.38      0.29      0.26      1046
weighted avg       0.35      0.28      0.24      1046

Evaluating endpoint-simple...
Unique true labels: {1, 2, 3, 4, 5}
Unique predicted labels: {1, 2, 3, 4, 5}
Accuracy for endpoint-simple: 28.68%
Confusion Matrix for endpo

In [54]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
from sagemaker.predictor import Predictor
predictor = Predictor(endpoint_name="endpoint-simple")
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: endpoint-simple
INFO:sagemaker:Deleting endpoint with name: endpoint-simple


In [55]:
predictor = Predictor(endpoint_name="endpoint-resnet")
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: endpoint-resnet
INFO:sagemaker:Deleting endpoint with name: endpoint-resnet


In [56]:
predictor = Predictor(endpoint_name="endpoint-hpo")
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: endpoint-hpo
INFO:sagemaker:Deleting endpoint with name: endpoint-hpo
